# Ventas con Incertidumbre

Usando datos desde [Kaggle, Rossman stores](https://www.kaggle.com/c/rossmann-store-sales/data).

Nuestro objetivo en este ejemplo es estimar las ventas promedio en función del día de la semana, con un análisis de incertidumbre derivado de dos formas: analíticamente y mediante realización aleatoria. Este ejemplo no pretende ser lo que le gustaría hacer en este escenario para responder a una pregunta simple, sino una buena práctica sobre cómo simular RV para estimar la incertidumbre, la pérdida de información que se produce si realiza aproximaciones analíticas, pero por qué pueden ser útiles.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, lognorm
df = pd.read_csv("sales.csv")
df.info()

In [ ]:
df.head()

In [ ]:
df2 = df.loc[(df.Open == 1), ["Store", "DayOfWeek", "Sales", "Customers"]]

In [ ]:
pd.plotting.scatter_matrix(df2.iloc[::100, :], figsize=(6, 6));

He aquí una forma muy breve de hacer las cosas. Agregue y regrese con una suposición gaussiana.

In [ ]:
df3 = df2.groupby("DayOfWeek").agg({"Sales": ["mean", "std"]})
df3

Pero esto no es tan educativo. En su lugar, tracemos una distribución, una por día de la semana, para la venta promedio de cualquier cliente dado (sobre todas las tiendas).

In [ ]:
for day, frame in df2.groupby("DayOfWeek"):
    plt.hist(frame.Sales / frame.Customers, bins=50, label=day, histtype='step', density=True)
plt.legend()
plt.xlabel("$ Spent")
plt.ylabel("P($)");

¡Parece un buen ajuste para un normal para mí! (No mires el domingo, que duele). Echemos un vistazo a la relación entre las medias y las desviaciones estándar de cada día.

In [ ]:
res = {}
for day, frame in df2.groupby("DayOfWeek"):
    data = frame.Sales / frame.Customers
    mean, std = data.mean(), data.std()
    res[day] = {"Gasto": (mean, std)}
    plt.scatter([mean], [std], label=day, s=30)
plt.legend();
plt.xlabel("Media Gasto")
plt.ylabel("Dispersión Gasto")
print(res)

Vaya, ¿qué está pasando el domingo? ¡No es un día rentable en absoluto! La relación casi lineal que vemos a la derecha sería coherente con la escala de la varianza en función del gasto. Es decir, si normalizamos el diferencial en términos de gasto, podría aplanarse. Solo como un lado interesante, en realidad no necesitamos usar esa información en los datos, ¡pero lo tenemos por si acaso!

Por ejemplo, si tuviéramos que usar esta información de datos, solo necesitaríamos la cantidad de clientes y el gasto medio (dos vectores de datos) en lugar de la cantidad de clientes, el gasto medio y la distribución del gasto. Porque podemos utilizar las relaciones existentes para predecir eso. Pero dado que nuestros datos se han extendido, ¡también podríamos usar eso!

Ahora, tenemos nuestra aproximación normal al gasto por cliente. Ahora necesitamos cuantificar el número de clientes en función del día. Visualicemos de nuevo.

In [ ]:
for day, frame in df2.groupby("DayOfWeek"):
    plt.hist(frame.Customers, bins=100, histtype="step", label=day, density=True)
plt.legend();
plt.xlabel("# Clientes")
plt.ylabel("P(Num)");

Maldito domingo, deja de arruinar todo. En este punto, estaría investigando el domingo más si solo tuviera los datos. ¿Diferencias de horario de apertura? ¿Descuentos el domingo? ¿Solo algunas tiendas abren los domingos en áreas más pobladas o áreas en áreas socioeconómicas más altas? Nos gustaría encontrar datos complementarios para profundizar en esto, lo que puede hacer fácilmente si está trabajando en un entorno empresarial, pero no puede hacerlo si está utilizando datos antiguos de código abierto.

Ahora, número de clientes, solo número positivo, contando valores reales... ¡suena como una buena aplicación para un registro normal para mí!

In [ ]:
fig, axes = plt.subplots(figsize=(10,6), nrows=2, ncols=4, sharey=True, sharex=True)
fig.subplots_adjust(wspace=0)

for ax, (day, frame) in zip(axes.flatten(), df2.groupby("DayOfWeek")):
    params = lognorm.fit(frame.Customers, scale=1000)
    ax.hist(frame.Customers, bins=100, density=True, histtype="step")
    xs = np.linspace(frame.Customers.min(), frame.Customers.max(), 100)
    ax.plot(xs, lognorm.pdf(xs, *params))
    ax.set_title(day)
    ax.set_xlim(0, 3000)
    res[day]["cust"] = list(params)
    print(day, params)

Entonces, con la excepción del domingo, realmente podríamos modelar *todas* estas distancias con una media de 0, un parámetro de forma de alrededor de 0,4 y una escala que varía ligeramente con cada día, generalmente alrededor de 700.

De nuevo, esto va en la línea de encontrar la información que es importante en los datos. Si descartamos el domingo y usamos la información de datos anterior, ahora podemos decir que las ganancias diarias se pueden predecir usando solo dos números: la escala de la norma logarítmica que representa la cantidad de clientes y el valor medio del gasto de cada cliente. Inferiríamos cualquier otro valor.

Comparemos esa inferencia de número reducido, con una inferencia completa, solo con el promedio con el que comenzamos.

In [ ]:
df2.groupby("DayOfWeek").agg({"Sales": ["mean", "std"], "Customers": ["mean", "std"]}).head()

In [ ]:
# Si quisiéramos dibujar valores simulados y ejecutarlos (en lugar de usar los datos reales)
percentiles = 100 * norm.cdf([-1, 0, 1])
n = 100000
# para cada día, muestree # clientes y muestree su gasto
for day, dic in res.items():
    spend = norm(*dic["spend"]).rvs(n)
    custs = lognorm(*dic["cust"]).rvs(n)
    profit = custs * spend
    vals = np.percentile(profit, percentiles)
    diff = np.diff(vals)
    print(f"Día {day} tiene benificios {vals[1]:.2f}(+{diff[1]:.2f})(-{diff[0]:.2f})")

¡Tenga en cuenta que estas estimaciones son más bajas! El uso de este método es menos sensible a los valores atípicos que aumentan las ganancias, y obviamente hemos perdido algo de información en nuestras aproximaciones.

Consideremos ahora el caso en que comprimimos los datos en solo dos números: número medio de clientes y gasto promedio.

In [ ]:
for day, dic in res.items():
    mean_spend = dic["spend"][0]
    num_cust = dic["cust"][2]
    
    spend = norm(mean_spend, mean_spend*0.25).rvs(n)
    custs = lognorm(0.4, 0, num_cust).rvs(n)
    profit = custs * spend
    vals = np.percentile(profit, percentiles)
    diff = np.diff(vals)
    print(f"Día {day} tiene beneficios {vals[1]:.2f}(+{diff[1]:.2f})(-{diff[0]:.2f})")

Así que puedes ver, aparte del domingo, esta es una aproximación bastante buena. Y si hace algo como esto, puede intentar responder más preguntas fácilmente. Imagine que ahora obtiene información de que en la semana anterior a las vacaciones la cantidad que la gente gasta se dispersa más. Por ejemplo, hay un aumento del 50 % en la dispersión del gasto y la gente también gasta en promedio un dólar más. ¿Cómo afectaría eso a las ganancias?

Usando la forma original, no es obvio cómo escribiríamos una estimación de una línea. Pero es fácil con nuestras aproximaciones: simplemente modifique las distribuciones de las que estamos dibujando.

In [ ]:
for day, dic in res.items():
    mean_spend = dic["spend"][0]
    num_cust = dic["cust"][2]
    
    spend = norm(mean_spend+1, mean_spend*0.25*1.5).rvs(n)
    custs = lognorm(0.4, 0, num_cust).rvs(n)
    profit = custs * spend
    vals = np.percentile(profit, percentiles)
    diff = np.diff(vals)
    print(f"Día {day} tiene beneficios {vals[1]:.2f}(+{diff[1]:.2f})(-{diff[0]:.2f})")

Tenga en cuenta que en este ejemplo no hay ninguna razón real por la que haríamos todo este esfuerzo para caracterizar nuestras distribuciones solo para tomar muestras de ellas nuevamente. Simplemente usaríamos los datos reales; tenemos suficientes muestras para hacerlo. O una muestra del PDF empírico si *realmente* queríamos muestrear. Este ejercicio simplemente pretende mostrarle cómo *puede* usar el muestreo de números aleatorios para estimar la incertidumbre. También significa que podemos responder preguntas más flexibles: el método original de una línea solo se adapta a los escenarios exactamente como los describen los datos.